In [45]:
import polars as pl
import os
from dotenv import load_dotenv
import mosqlient as mosq
import subprocess

# Load environment variables from .env file
load_dotenv()

# Access the environment variables
api_key = os.getenv('API_KEY')

def get_git_commit_hash():
    try:
        commit_hash = subprocess.check_output(['git', 'rev-parse', 'HEAD'], cwd=os.path.dirname(os.path.abspath(__file__))).decode('ascii').strip()
    except Exception:
        # fallback: try from current working directory
        try:
            commit_hash = subprocess.check_output(['git', 'rev-parse', 'HEAD']).decode('ascii').strip()
        except Exception:
            commit_hash = None
    return commit_hash

COMMIT = get_git_commit_hash()

In [8]:
name = "Chronos-Bolt"
description = "Chronos-Bolt model fine-tuned for 2025 Sprint"
repository = "https://github.com/davibarreira/jbd-mosqlimate-sprint"
implementation_language = "Python"
disease = "dengue"
temporal = True
spatial = False
categorical = False
adm_level = 1
time_resolution = "week"
sprint = True

model = mosq.upload_model(
    api_key=api_key,
    name=name,
    description=description,
    repository=repository,
    implementation_language=implementation_language,
    disease=disease,
    temporal=temporal,
    spatial=spatial,
    categorical=categorical,
    adm_level=adm_level,
    time_resolution=time_resolution,
    sprint=sprint
)
model.dict()

{'id': 133,
 'name': 'Chronos-Bolt',
 'description': 'Chronos-Bolt model fine-tuned for 2025 Sprint',
 'author': {'user': {'name': 'Davi Sales Barreira',
   'username': 'davibarreira'},
  'institution': 'FGV'},
 'repository': 'https://github.com/davibarreira/jbd-mosqlimate-sprint',
 'implementation_language': {'language': 'Python'},
 'disease': 'dengue',
 'categorical': False,
 'spatial': False,
 'temporal': True,
 'adm_level': 1,
 'time_resolution': 'week',
 'sprint': True}

In [ ]:
model_id = 133


In [ ]:
for i in [1,2,3]:
    submission = pl.read_parquet(f'../data/5_predictions/submission{i}.parquet')
    for uf in submission['uf'].unique():
        df = submission.filter(pl.col('uf') == uf).drop('uf').to_pandas()

        res = mosq.upload_prediction(
                    model_id = model_id, 
                    description = f'Validation set {i} for {uf} using Chronos-Bolt', 
                    commit = COMMIT,
                    predict_date = '2025-07-31', 
                    prediction = df,
                    adm_1=uf,
                    api_key = api_key)